In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

%pylab inline

train = pd.read_csv('../data/train_preperd01.csv')

Populating the interactive namespace from numpy and matplotlib


In [2]:
train.shape

(1452, 187)

In [3]:
def backwardElimination(X, Y):
    for i in np.arange(0.9,0,-0.1):
        numVars = len(X.columns.tolist())
        d=[]
        regressor_OLS = sm.OLS(Y, X).fit()
        for j in range(numVars):
            if regressor_OLS.pvalues[j] > i:
                d.append(X.columns[j])
        for t in d:
            X=X.drop(t, axis=1)
    
    regressor_OLS = sm.OLS(Y, X).fit()
    while max(regressor_OLS.pvalues > i):
        numVars = len(X.columns.tolist())
        d=[]
        for j in range(numVars):
            if regressor_OLS.pvalues[j] > i:
                d.append(X.columns[j])
        for t in d:
            X=X.drop(t, axis=1)
        regressor_OLS = sm.OLS(Y, X).fit()
    
    print(regressor_OLS.summary())
    
    return regressor_OLS, X.columns.tolist()

def predicting(train, list_to_remove):
    columns_to_keep=train.columns.tolist()
    for i in list_to_remove:
        columns_to_keep.remove(i)

    X=train[columns_to_keep]
    y=train.Log_sale_price
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    regressor_OLS = sm.OLS(y_train,x_train).fit()
    print(regressor_OLS.summary())
    
    y_pred=regressor_OLS.predict(x_test)
    mse = mean_squared_error(y_pred,y_test)
    rmse = np.sqrt(mse)
    print("\n rmse on all var's is: ", rmse)   
    
    regressor, ctp=backwardElimination(x_train,y_train)
    
    y_pred2=regressor.predict(x_test[ctp])
    mse2 = mean_squared_error(y_pred2,y_test)
    rmse2 = np.sqrt(mse2)
    print("\n rmse on best var's is: ", rmse2)

    return y_pred, y_pred2,y_test,ctp,regressor

def plotting_pred(y_pred,y_test,y_pred2):
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(1,2,1)
    ax.scatter(y_pred,y_test, s=5)
    ax.set_title('prediction of log10 prices')
    ax.set_xlabel('predicted price')
    ax.set_ylabel('real price')

    bx = fig.add_subplot(1,2,2)
    bx.scatter(y_pred2,y_test, s=5)
    bx.set_title('prediction of log10 prices')
    bx.set_xlabel('predicted price')
    bx.set_ylabel('real price')

    fig.show()

In [4]:
y_pred, y_pred2,y_test,ctp,regressor=predicting(train,['Log_sale_price','SaleDate', 'Id','YrSold','MoSold'])

                            OLS Regression Results                            
Dep. Variable:         Log_sale_price   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     68.06
Date:                Sun, 18 Nov 2018   Prob (F-statistic):               0.00
Time:                        14:21:56   Log-Likelihood:                 1722.6
No. Observations:                1016   AIC:                            -3101.
Df Residuals:                     844   BIC:                            -2254.
Df Model:                         171                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Un

                            OLS Regression Results                            
Dep. Variable:         Log_sale_price   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.920
Method:                 Least Squares   F-statistic:                     162.2
Date:                Sun, 18 Nov 2018   Prob (F-statistic):               0.00
Time:                        14:21:58   Log-Likelihood:                 1671.9
No. Observations:                1016   AIC:                            -3198.
Df Residuals:                     943   BIC:                            -2838.
Df Model:                          72                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
LotArea                1.618e-

In [6]:
test = pd.read_csv('../data/test_preperd01.csv')

In [13]:
test.loc[test[test.BsmtFullBath.isnull()==True].index.tolist(),'BsmtFullBath']=0
test_X=test[ctp]
test_X.loc[:,test_X.isnull().any()].columns.tolist()

[]

In [14]:
y_pred_test=regressor.predict(test_X)

y_pred_test.isnull().any()

False

In [15]:
SalePrice=pd.Series(10**y_pred_test, name='SalePrice')

test=test.join(SalePrice)

submission = test.loc[:, ['Id', 'SalePrice']].copy()
submission.set_index('Id', inplace=True)

submission.to_csv('../data/model09.csv')